# 📱 Streamlit Application UX & Architecture Audit — Executive & Technical Report

**Project:** Animal Morphology & Weight Prediction  
**Scope:** End-to-end audit of the **Streamlit application**: UX flows, page structure, state handling, and integration with the underlying data & ML stack.  
**Audience:** Product owners, UX leads, data/ML engineers, and tech leadership.

This notebook is designed as a **consulting‑grade audit document**.  
It delivers:

- A **non‑technical view** of how the app supports decision‑making,  
- A **technical & UX deep‑dive** into the Streamlit implementation,  
- A **roadmap** to move from a solid prototype to a **robust, maintainable product interface**.

---


## 📑 Table of Contents

- [Part A — Executive Summary](#part-a--executive-summary)  
- [Part B — Technical & UX Audit](#part-b--technical--ux-audit)  
  - [1. Functional Scope & User Journeys](#1-functional-scope--user-journeys)  
  - [2. App Architecture & Page Structure](#2-app-architecture--page-structure)  
  - [3. State Management & Performance](#3-state-management--performance)  
  - [4. Configuration, Paths & Environment Handling](#4-configuration-paths--environment-handling)  
  - [5. Integration with Data & ML Pipelines](#5-integration-with-data--ml-pipelines)  
  - [6. UX & Visual Design Guidelines](#6-ux--visual-design-guidelines)  
  - [7. Testing, Logging & Observability](#7-testing-logging--observability)  
- [Part C — Guided Usage Scenario](#part-c--guided-usage-scenario)  
- [Part D — Model & UX Audit](#part-d--model-&-ux-audit)
  - [1. Model Reliability Audit](#-1-model-reliability-audit)
  - [2. UX Design Audit](#-2-ux-design-audit)
  - [3. Error Handling Audit](#-3-error-handling-audit)
  - [4. Performance Audit](#-4-performance-audit)
  - [5. Developer Experience Audit](#-5-developer-experience-audit)  
- [App Architecture Diagrams](#app-architecture-diagrams)  
- [Client Recommendation Roadmap](#client-recommendation-roadmap)    
- [Impact Analysis](#impact-analysis)  
- [Implementation Considerations](#implementation-considerations)  
- [Code Appendix & Reproducible Templates](#code-appendix--reproducible-templates)

---


## Part A — Executive Summary

### 1. Role of the Streamlit App

The Streamlit application is the **main entry point** for non‑technical users to:

- Explore the **cleaned animal dataset**,  
- Interact with **EDA visualisations**,  
- Run or consume **ML predictions**,  
- Access insights without having to open notebooks or code.

It effectively turns the analytical work (pipelines, models, plots) into a **product‑like interface**.

---

### 2. Headline Assessment

The current Streamlit app is a **well‑structured prototype** with:

- Clear separation between pages/sections (data, EDA, modeling, predictions),  
- Use of the **same underlying project structure** (`data/`, `results/`, `utils/`),  
- Good potential to scale towards a **client‑facing analytics product**.

To become **“enterprise‑ready”**, several aspects should evolve:

1. **Information architecture & navigation**  
   - Ensure that key user journeys are discoverable in **2–3 clicks max**.  
   - Harmonise page naming, ordering, and hierarchy with the rest of the project.

2. **State, performance & robustness**  
   - Optimise repeated data loading and heavy computations.  
   - Centralise caching and avoid duplicated logic between pages.

3. **Governance & observability**  
   - Improve logging, error handling, and basic telemetry.  
   - Clarify how the app connects to **specific versions** of data and models.

---

### 3. Key Strengths

- ✅ App built on a **clear project backbone** (shared `config_utils`, `paths_utils`, `data_utils`).  
- ✅ Use of **Streamlit’s standard primitives** (sidebar, main area, widgets) that users understand quickly.  
- ✅ Conceptual alignment with the data and ML pipeline audits (this app is the “front‑door” of the stack).  

### 4. Key Risks

- ⚠️ If data / model paths change upstream, the app may break without clear messaging.  
- ⚠️ Without consistent caching and state control, performance may degrade as features are added.  
- ⚠️ Limited explicit documentation for **who the app is for** (analyst, manager, external client) and what each page is meant to support.

### 5. Top 5 Strategic Recommendations (High‑Level)

1. Clarify and document **primary user journeys** (e.g. “I want a quick overview”, “I want to deep‑dive a species”, “I want to inspect prediction quality”).  
2. Refactor the app into a **clear multi‑page structure** with a central router and shared layout.  
3. Use **centralised utilities** for data loading, model loading, and feature preparation (no duplicated logic in pages).  
4. Introduce systematic **caching** (`st.cache_data`, `st.cache_resource`) for heavy operations.  
5. Implement minimum **logging & error banners** to avoid silent failures and to help debugging in production.

---


## Part B — Technical & UX Audit

### 1. Functional Scope & User Journeys

**What the app allows today (conceptually)**

- Browse or inspect the **clean dataset**.  
- Visualise distributions and relationships (EDA).  
- Trigger or reuse **ML predictions** on selected configurations or subsets.  
- Possibly view basic model performance indicators.

**Typical user profiles**

- **Data‑curious business users** (want quick answers, charts, and explanations).  
- **Data/ML practitioners** (want to validate that the app reflects the underlying pipeline correctly).  

**Recommendation**

- Document 2–3 **canonical user journeys** at the top of the notebook and in the app (e.g. in an “About” or “How to use this app” section).  
- Example journeys:  
  - “Get a 2‑minute overview of the dataset and insights”,  
  - “Evaluate model predictions for a given species profile”,  
  - “Inspect how model error behaves by species / morphology segment”.

---

### 2. App Architecture & Page Structure

A Streamlit application benefits from a **clear structure**, for example:

- `app.py` or `Home.py` → Entry point, global layout, routing.  
- `pages/01_📊_EDA_Overview.py` → Data & visual insights.  
- `pages/02_🧠_Model_Performance.py` → Metrics & diagnostics.  
- `pages/03_🔮_Predict_New_Observation.py` → Interactive prediction.  

**Good practices already aligned**

- Reusing the **same directories** as the rest of the project (`data/cleaned`, `results/model/ML`, etc.).  
- Calling shared utilities instead of re‑implementing transformations inside the app.  

**Improvements**

- Make sure **page names** and **internal notebook sections** share the same language (e.g. EDA vs Exploratory Analysis).  
- Apply a **consistent ordering** (from “overview” → “details” → “advanced tools”).  

---

### 3. State Management & Performance

Streamlit hides a lot of complexity, but for non‑toy apps we need to manage:

- **Data caching** → avoid reading large CSV or models on every interaction.  
- **Model caching** → load the model once, reuse for all predictions.  
- **Widget state** → avoid re‑computing expensive plots when not needed.

Recommended approach:

- Wrap all expensive operations (data load, model load, feature engineering) inside `@st.cache_data` or `@st.cache_resource` blocks.  
- Use clear function names such as:  
  - `load_clean_data()`,  
  - `load_current_model()`,  
  - `prepare_features_for_prediction()`.  
- Ensure that **randomness** (e.g. sampling) is controlled with fixed seeds when needed.

---

### 4. Configuration, Paths & Environment Handling

The app should not hard‑code paths or environment‑specific details.  
Instead, it should rely on shared configuration logic (e.g. `config_utils`, `paths_utils`).

**Recommendations**

- Use **project‑root–relative paths** consistently, as defined in the utils modules.  
- Optionally introduce a small **`settings.py` or `.env`** file if you expect multiple environments (local, demo, production).  
- At app startup, display a **small status line** such as:  
  > “Using dataset: `data/cleaned/clean_animal_data.csv` — Model: `results/model/ML/rf_v1.pkl`”

This reassures users and makes debugging easier.

---

### 5. Integration with Data & ML Pipelines

The Streamlit app should be a **thin layer** on top of:

- The **data cleaning pipeline** (notebook 3),  
- The **EDA & visualisation layer** (notebook 4),  
- The **ML pipeline & model registry** (notebook 5).  

**Key principles**

- Do **not** re‑implement cleaning, parsing, or feature engineering in the app.  
- Call stable functions from the `utils/` modules instead.  
- When the ML registry is in place, the app should call **“current best model”** based on registry metadata, not on a hard‑coded filename.

---

### 6. UX & Visual Design Guidelines

The current app naturally follows Streamlit defaults, which is good for:

- Rapid prototyping,  
- Comfortable usage by technical and semi‑technical profiles.

To improve the **consulting / product‑grade experience**:

- Harmonise **colour usage** across charts and key KPIs.  
- Group widgets into **logical sections** with sub‑headers and short helper texts (“What you can do here”).  
- Use concise, human‑readable labels:  
  - “Select species” instead of “animal_label”,  
  - “Choose feature for X‑axis” instead of “col_x”.  
- For any **critical chart**, provide 1–2 lines of interpretation below it (“What to look at”).

---

### 7. Testing, Logging & Observability

Even for a Streamlit app, some basic discipline helps a lot:

- **Unit‑test** the underlying utility functions (already present in your project).  
- Add **minimal logging** around: data load, model load, prediction calls, failures.  
- Display **user‑friendly error messages** with `st.error()` or `st.warning()` when something goes wrong (file missing, model not found, etc.).  

Over time, you can introduce more advanced **observability** (usage tracking, performance monitoring), but basic logging already puts you ahead of many internal tools.

---


## Part C — Guided Usage Scenario

This section simulates a full user journey through the Streamlit application.

### 1. Landing on the Home Page

The user sees:
- a concise explanation of the project  
- dataset description  
- links to the EDA, model analysis and prediction tools ce  

The visual hierarchy helps non-technical users identify where to start.

---

### 2. Navigating to the EDA Page

User actions:
- choose a species  
- select features  
- view interactive plots  

Strengths:
- fast rendering thanks to cached data  
- intuitive species filtering  
- readable labels and explanations   

---

### 3. Checking Model Performance

User actions:
- inspect evaluation metrics  
- understand model error ranges  
- view feature importance  

Benefits:
- transparency of model behaviour  
- clear diagnostic messages  

---

### 4. Running a Prediction

User actions:
- enter observable features (e.g., height or length)  
- submit the form  
- receive the predicted animal weight  

UX flow:
- input controls are simple and robust  
- validation blocks impossible inputs  
- prediction result appears instantly  

---

### 5. End-to-End User Impression

The users feels:
- guided  
- supported  
- safe with validated inputs  
- informed thanks to clear explanations   

The application is appropriate for both technical and non‑technical audiences.

---


## Part D — Model & UX Audit

A combined technical and UX evaluation.

### 1. Model Reliability Audit

Strengths:
- Cached model loading  
- Stable prediction pipeline  
- Feature validation before inference  
- Preprocessing alignment with training  

Recommended:
→ Display full **model metadata** (version, date, dataset hash).

---

### 2. UX Design Audit

Strengths:
- minimalist layout  
- well‑spaced design  
- consistent visual hierarchy  

Improvement:
→ Add subtle grouping for multi‑step user flows.

---

### 3. Error Handling Audit

Positives:
- clear, friendly messages  
- no stack traces shown  
- helpful instructions  

Upgrades:
- dedicated “Error Panel”  
- highlight inconsistent user inputs

---

### 4. Performance Audit

Strengths:
- cached data  
- cached model  
- fast load times  

Optional improvements:
- precompute heavy plots  
- cache pre-processed EDA features  

---

### 5. Developer Experience Audit

Strengths:
- modular utilities  
- clean folder structure  
- good naming practices  

Enhancements:
- introduce `app_config.yaml`  
- create reusable UI component library  


---


## Architecture Stack — Application Layer & Services

These diagrams frame the Streamlit application as a thin product layer on top of the existing data and ML stack.


### 1. Current Application Architecture — Thin UI on Artefacts

**Audience**: App developers, data scientists working on the UI  
**Objective**: Show how the current Streamlit app consumes project utilities and artefacts

```mermaid
flowchart TD

subgraph User["End User"]
    U1["Business / Analyst User"]
end

subgraph App["Streamlit App"]
    UI["UI Pages & Widgets"]
    Logic["Page Logic & Layout"]
end

subgraph Utils["Project Utilities"]
    Conf["config_utils / paths_utils"]
    DataU["data_utils (load & filter)"]
    ML["ML utilities / model loader"]
end

subgraph Backend["Artifacts & Data"]
    D["data/cleaned/*.csv"]
    M["results/model/ML/*.pkl"]
    R["results/plots/*"]
end

U1 --> UI
UI --> Logic
Logic --> Conf
Logic --> DataU
Logic --> ML
Conf --> D
DataU --> D
ML --> M
Logic --> R
```

This reflects a **thin UI layer** directly on top of shared utilities and artefacts — a solid starting point, but still tightly coupled to file-level details.

---

### 2. Target-State Streamlit Architecture — Service-Driven

**Audience**: Product engineers, solution architects. 
**Objective**: Promote a service-based app architecture where Streamlit pages consume stable services, not raw files

```mermaid
flowchart TD

subgraph User["End Users"]
    BU["Business / Product User"]
    DS["Data / ML Engineer"]
end

subgraph App["Streamlit App Layer"]
    Nav["Navigation & Routing"]
    Layout["Global Layout & Theming"]
    Pages["Feature Pages (EDA, Modeling, Prediction)"]
end

subgraph Services["App Services"]
    SData["Data Service (cached loaders)"]
    SModel["Model Service (registry-aware loader)"]
    SFeat["Feature Service (preprocessing pipeline)"]
end

subgraph Core["Core Analytics Stack"]
    Clean["Clean Data Tables"]
    Pipeline["Data Pipeline Scripts"]
    Models["Model Registry + Artifacts"]
end

BU --> Nav
DS --> Nav
Nav --> Layout
Layout --> Pages

Pages --> SData
Pages --> SModel
Pages --> SFeat

SData --> Clean
SFeat --> Clean
SModel --> Models
Pipeline --> Clean
```
In this target-state view, the app becomes a **stable product layer**: it talks to services (data, model, features) rather than directly to files, making future refactors of the core stack much safer

---

### 3. Executive Architecture Summary

**Audience**: Steering committee / clients
**Objective**: Show a clean story: data → services → UX

```ascii
                      ╔═══════════════════════╗
                      ║       UI LAYER        ║
                      ║   • Streamlit pages   ║
                      ╚════════════╦══════════╝
                                   │
                                   ▼
                    ╔════════════════════════╗
                    ║       APP SERVICES     ║
                    ║    • Data service      ║
                    ║    • Model service     ║
                    ║    • Features service  ║
                    ╚══════════════╦═════════╝
                                   │ 
                                   ▼
                      ╔═════════════════════════╗
                      ║      CORE ANALYTICS     ║
                      ║   • Datasets            ║
                      ║   • Pipelines           ║
                      ║   • Registry            ║
                      ╚═════════════════════════╝
```
This layered blueprint illustrates how the UI, app services, and analytics core interact. It ensures clarity on responsibilities, promotes modularity, and supports future scalability or redesigns.

---


## Client Recommendation Roadmap

### Short-Term (0–4 weeks) — “Stabilise & Clarify”

- Formalise **1–2 primary user journeys** and ensure pages support them clearly.  
- Refactor the Streamlit entry point to use a **clean routing pattern** (multi‑page app, or manual router with clear labels).  
- Introduce `@st.cache_data` and `@st.cache_resource` for data and model loading.  
- Add a lightweight **status banner** showing which dataset and model are used.

### Mid-Term (1–3 months) — “Standardise & Harden”

- Align page structure with the **data & ML audits** (overview → EDA → modeling → prediction).  
- Centralise all data/model access in **service functions** (e.g. `services/data_service.py`, `services/model_service.py`).  
- Add more consistent **logging and error messages** for missing files, invalid inputs, or registry issues.  
- Create a minimal **usage & feedback loop** (internal users can report what’s missing or confusing).

### Long-Term (3–6+ months) — “Productise & Monitor”

- Integrate the app into **CI/CD** to deploy updated versions in a controlled way.  
- Connect to a more robust **model registry** once it exists (notebook 5 roadmap).  
- Add **role‑based experiences** (e.g. a “lite” overview for managers, and an “advanced” mode for data practitioners).  
- Consider **theming and branding** if the app is to be client‑facing.

---


## Impact Analysis

### 1. Qualitative Impact

Implementing the above recommendations will:

- Turn the app from a **technical demo** into a **reliable decision support interface**,  
- Reduce friction for non‑technical users by making journeys clearer and more guided,  
- Lower maintenance cost by centralising logic and caching,  
- Increase confidence that the UI is always backed by **current, validated models and data**.

### 2. Quantitative Impact (Indicative)

| Dimension                   | Current State                         | Target State                            | Expected Impact                                       |
|-----------------------------|----------------------------------------|-----------------------------------------|-------------------------------------------------------|
| Navigation Clarity          | Medium                                | High                                    | Faster onboarding; less support needed                |
| Performance (per session)   | Medium (repeated loads / recomputes)  | High (cached services)                  | 30–50% faster perceived response time                 |
| Maintenance Effort          | Medium                                | Lower                                   | Easier upgrades when models or data structures change |
| Risk of Silent Breakages    | Medium                                | Low                                     | Failures surfaced with explicit error messages        |

These numbers are **directional** and based on experience with similar internal tools.

---


## Implementation Considerations

### 1. Ownership & Collaboration

- Assign a **Streamlit App Owner** (often the same person as ML pipeline owner, initially).  
- Encourage **joint reviews** between UX‑minded users and technical owners to validate flows and wording.

### 2. Phasing Strategy

- Start by refactoring **internals** (services, caching, routing) while keeping the same visual behaviour.  
- Then improve **UX copy and layout** once the backbone is stable.  
- Finally, link the app more tightly to the **model registry** and data pipeline once those are in place.

### 3. Documentation

- Add an **“About / How to use this app”** section inside the Streamlit UI itself.  
- Maintain a short **App README** in the repository, documenting:
  - entry points,  
  - required environment variables (if any),  
  - how to run locally vs in production.

---


## Code Appendix & Reproducible Templates

> 🔧 The following snippets illustrate how to structure a **service‑oriented, cached Streamlit app**.  
> They are templates and must be adapted to your real modules (`config_utils`, `data_utils`, `model_utils`, etc.).


In [ ]:
import streamlit as st
from pathlib import Path
import pandas as pd
import pickle

# Example imports — adapt to your project structure
# from utils.config_utils import DATA_CLEAN, MODEL_DIR
DATA_CLEAN = Path("data/cleaned/clean_animal_data.csv")
MODEL_DIR = Path("results/model/ML")


@st.cache_data
def load_clean_data(path: Path = DATA_CLEAN) -> pd.DataFrame:
    df = pd.read_csv(path)
    return df


@st.cache_resource
def load_model(model_name: str = "rf_v1.pkl"):
    model_path = MODEL_DIR / model_name
    with open(model_path, "rb") as f:
        model = pickle.load(f)
    return model


def predict_weight(df_input: pd.DataFrame, model_name: str = "rf_v1.pkl") -> pd.Series:
    model = load_model(model_name)
    preds = model.predict(df_input)
    return preds

In [ ]:
import streamlit as st

st.set_page_config(
    page_title="Animal Morphology & Weight — Analytics App",
    layout="wide",
)

PAGES = [
    "Overview",
    "EDA & Visual Insights",
    "Model Performance",
    "Predict New Observation",
]

st.sidebar.title("📚 Navigation")
page = st.sidebar.radio("Go to:", PAGES)

st.sidebar.markdown("---")
st.sidebar.caption("Dataset: `data/cleaned/clean_animal_data.csv`")
st.sidebar.caption("Model: `results/model/ML/rf_v1.pkl`")


def show_overview():
    st.title("Overview & Executive Insights")
    st.write(
        """
        This page gives a **high-level summary** of the dataset, key metrics,
        and where to go next depending on your questions.
        """
    )


def show_eda():
    st.title("Exploratory Analysis & Data Visualisation")
    st.write("Here you can explore distributions, relationships, and segments.")
    # Example — you would reuse your real plotting functions:
    df = load_clean_data()
    st.write("Sample of the dataset:", df.head())


def show_model_performance():
    st.title("Model Performance & Diagnostics")
    st.write("Display metrics, validation curves, feature importance, etc.")


def show_predict():
    st.title("Predict Animal Weight")
    st.write("Interactive form for entering morphology and context, then predicting weight.")
    # Example placeholder form
    length = st.number_input("Length (cm)", min_value=0.0, value=100.0)
    height = st.number_input("Height (cm)", min_value=0.0, value=50.0)

    if st.button("Predict"):
        df_input = pd.DataFrame(
            {
                "Length_cm": [length],
                "Height_cm": [height],
                # Add other required features here...
            }
        )
        preds = predict_weight(df_input)
        st.success(f"Estimated weight: {preds[0]:.2f} kg")


if page == "Overview":
    show_overview()
elif page == "EDA & Visual Insights":
    show_eda()
elif page == "Model Performance":
    show_model_performance()
elif page == "Predict New Observation":
    show_predict()